# Model Datasets

Datasets used for parameterization and backtesting

Sections based on rows of `20220721_Fei Protocol Model Datasets_vDraft` spreadsheet

In [ ]:
import asyncio

import matplotlib.pyplot as plt
import numpy as np
import toolplot

from ctc import evm
from ctc import rpc
from ctc.protocols import aave_v2_utils
from ctc.protocols import compound_utils
from ctc.protocols import chainlink_utils
from ctc.protocols import fei_utils
from ctc.protocols.fei_utils import fei_psms
from ctc.protocols import rari_utils
from ctc.toolbox import plot_utils

%matplotlib inline

In [ ]:
toolplot.setup_plot_formatting()

In [ ]:
n_samples = 200

latest_block_number = await evm.async_get_latest_block_number()

## Protocol, PCV, and FEI Distribution

In [ ]:
fei_analytics_payload = await fei_utils.async_create_payload(
    timescale='180d,2d',
)

In [ ]:
fei_analytics_payload['data'].keys()

#### Historical PSM FEI minting / redemption rates

In [ ]:
mints = await fei_psms.async_get_fei_psm_mints()

mints

In [ ]:
redemptions = await fei_psms.async_get_fei_psm_redemptions()

redemptions

In [ ]:
plt.plot(mints['arg__amountFeiOut'], '.b', alpha=0.3, label='mint')
plt.plot(mints['arg__amountFeiOut'], '-b', alpha=0.1)

plt.plot(redemptions['arg__amountFeiIn'], '.r', alpha=0.3, label='redemption')
plt.plot(redemptions['arg__amountFeiIn'], '-r', alpha=0.1)

plt.yscale('log')
plt.ylim([10, 100e6])

await plot_utils.async_xtick_block_dates()
toolplot.add_tick_grid()
plt.legend()
toolplot.format_yticks()
plt.title('FEI Mints and Redemptions')
plt.show()

In [ ]:
markersize = 10

mask = mints['token'] == 'DAI'
plt.plot(mints['token_per_fei'][mask], '.', markersize=markersize, color='purple', label='mint, DAI per FEI')
plt.plot(mints['token_per_fei'][mask], '-', color='purple', alpha=0.2)

mask = mints['token'] == 'LUSD'
plt.plot(mints['token_per_fei'][mask], '.', markersize=markersize, color='red', label='mint, LUSD per FEI')
plt.plot(mints['token_per_fei'][mask], '-', color='red', alpha=0.2)

mask = redemptions['token'] == 'DAI'
plt.plot(redemptions['token_per_fei'][mask], '.', markersize=markersize, color='orange', label='redemption, DAI per FEI')
plt.plot(redemptions['token_per_fei'][mask], '-', color='orange', alpha=0.2)

mask = redemptions['token'] == 'LUSD'
plt.plot(redemptions['token_per_fei'][mask], '.', markersize=markersize, color='teal', label='redemption, LUSD per FEI')
plt.plot(redemptions['token_per_fei'][mask], '-', color='teal', alpha=0.2)

await plot_utils.async_xtick_block_dates()
toolplot.add_tick_grid()
plt.legend(loc='lower left')
toolplot.format_yticks()
plt.title('Minting and Redemption Rates')
plt.ylabel('token per FEI')
plt.show()

#### Historical PCV distribution across all assets and locations

In [ ]:
for asset, asset_data in fei_analytics_payload['data']['pcv_by_asset']['metrics'].items():
    plt.plot(fei_analytics_payload['block_numbers'], asset_data['values'], label=asset)
    
await plot_utils.async_xtick_block_dates()
toolplot.add_tick_grid()
plt.legend()
toolplot.format_yticks()
plt.title('PCV by Asset')
plt.show()

#### Historical protocol-owned FEI distribution

In [ ]:
for platform, pfei in fei_analytics_payload['data']['pfei_by_platform']['metrics'].items():
    if platform in ['Fei', 'Other', 'OA Account']:
        continue
    plt.plot(fei_analytics_payload['block_numbers'], pfei['values'], label=platform)

await plot_utils.async_xtick_block_dates()
toolplot.add_tick_grid()
plt.legend()
toolplot.format_yticks()
plt.title('pFEI Plaform Deployments')
plt.show()

#### Historical user-circulating FEI distribution

In [ ]:
for key, value in fei_analytics_payload['data']['circulating_fei']['metrics'].items():
    plt.plot(fei_analytics_payload['block_numbers'], value['values'], label=key)

await plot_utils.async_xtick_block_dates()
toolplot.add_tick_grid()
plt.legend()
toolplot.format_yticks()
plt.title('Circulating FEI')
plt.show()

## Asset Prices

#### Historical benchmark / basket for Stable Asset price

In [ ]:
dai_usd = await chainlink_utils.async_get_feed_data('DAI_USD', interpolate=True)

dai_usd

In [ ]:
lusd_usd = await chainlink_utils.async_get_feed_data('LUSD_USD', interpolate=True)

lusd_usd

In [ ]:
usdc_usd = await chainlink_utils.async_get_feed_data('USDC_USD', interpolate=True)

usdc_usd

In [ ]:
plt.plot(dai_usd, color='orange', label='DAI_USD')
plt.plot(lusd_usd, color='teal', label='LUSD_USD')
plt.plot(usdc_usd, color='dodgerblue', label='USDC_USD')

await plot_utils.async_xtick_block_dates()
toolplot.add_tick_grid()
plt.legend()
toolplot.format_yticks(toolstr_kwargs={'prefix': '$'})
plt.title('Stable Asset Prices')
plt.show()

#### Historical ETH price

In [ ]:
eth_usd = await chainlink_utils.async_get_feed_data('ETH_USD', interpolate=True)

eth_usd

In [ ]:
plt.plot(eth_usd, color='purple', label='ETH_USD')

await plot_utils.async_xtick_block_dates()
toolplot.add_tick_grid()
plt.legend()
toolplot.format_yticks(toolstr_kwargs={'prefix': '$'})
plt.title('ETH Price')
plt.show()

#### Historical FEI price

In [ ]:
fei_usd = await chainlink_utils.async_get_feed_data('FEI_USD', interpolate=True)

fei_usd

In [ ]:
plt.plot(fei_usd, color='green', label='FEI_USD')

await plot_utils.async_xtick_block_dates()
toolplot.add_tick_grid()
plt.legend()
toolplot.format_yticks(toolstr_kwargs={'prefix': '$'})
plt.title('FEI Price')
plt.show()

## Asset Yield

#### Historical benchmark / basket for Stable Asset yield

In [ ]:
# compound dai

cdai = '0x5d3a536e4d6dbd6114cc1ead35777bab948e3643'
cdai_creation_block = await evm.async_get_contract_creation_block(cdai)
cdai_blocks = np.linspace(cdai_creation_block, latest_block_number, n_samples).astype(int)
cdai_yields = await compound_utils.async_get_supply_apy_by_block(ctoken=cdai, blocks=cdai_blocks)

In [ ]:
# compound usdc

cusdc = '0x39aa39c021dfbae8fac545936693ac917d5e7563'
cusdc_creation_block = await evm.async_get_contract_creation_block(cdai)
cusdc_blocks = np.linspace(cusdc_creation_block, latest_block_number, n_samples).astype(int)
cusdc_yields = await compound_utils.async_get_supply_apy_by_block(ctoken=cusdc, blocks=cusdc_blocks)

In [ ]:
usdc = '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48'
dai = '0x6b175474e89094c44da98b954eedeac495271d0f'
fei = '0x956f47f50a910163d8bf957cf5846d573e7f87ca'

aave_v2_creation_block = await evm.async_get_contract_creation_block(
    aave_v2_utils.aave_interest_rates.aave_lending_pool,
)
aave_sample_blocks = np.linspace(aave_v2_creation_block, latest_block_number, n_samples).astype(int)

# aUSDC
ausdc_rates = await aave_v2_utils.async_get_interest_rates_by_block(
    token=usdc,
    blocks=aave_sample_blocks,
)
ausdc_yields = ausdc_rates['supply_apy']

# aDAI
adai_rates = await aave_v2_utils.async_get_interest_rates_by_block(
    token=dai,
    blocks=aave_sample_blocks,
)
adai_yields = adai_rates['supply_apy']

In [ ]:
linewidth = 2
plt.plot(cdai_blocks, cdai_yields, color='orange', label='cDAI', linewidth=linewidth)
plt.plot(cusdc_blocks, cusdc_yields, color='green', label='cUSDC', linewidth=linewidth)
plt.plot(aave_sample_blocks, adai_yields, color='red', label='aDAI', linewidth=linewidth)
plt.plot(aave_sample_blocks, ausdc_yields, color='purple', label='aUSDC', linewidth=linewidth)


await plot_utils.async_xtick_block_dates()
toolplot.add_tick_grid()
plt.legend()
toolplot.format_yticks(toolstr_kwargs={'percentage': True})
plt.ylabel('yield')
plt.title('Historical Stable Yields')
plt.show()

#### Historical ETH staking yield

In [ ]:
# compute staking rates of staking via lido

wsteth = '0x7f39c581f595b53c5cb19bd0b3f8da6c935e2ca0'

# compute timepoint samples
wsteth_creation_block = await evm.async_get_contract_creation_block(wsteth)
wsteth_blocks = np.linspace(wsteth_creation_block, latest_block_number, n_samples).astype(int)
wsteth_block_timestamps = await evm.async_get_block_timestamps(wsteth_blocks)
wsteth_block_timestamps = np.array(wsteth_block_timestamps)

# compute growth at each timepoint
steth_per_token = await rpc.async_batch_eth_call(
    to_address=wsteth,
    function_name='stEthPerToken',
    block_numbers=wsteth_blocks,
    provider={'chunk_size': 1},
)
steth_per_token = np.array(steth_per_token)

# convert to apy
wsteth_time_intervals = wsteth_block_timestamps[1:] - wsteth_block_timestamps[:-1]
wsteth_growth_intervals = steth_per_token[1:] / steth_per_token[:-1]
seconds_per_year = 86400 * 365
years_per_interval = wsteth_time_intervals / seconds_per_year
wsteth_apy = np.exp(np.log(wsteth_growth_intervals) / years_per_interval)

# plot
plt.plot(wsteth_blocks[1:], wsteth_apy - 1, linewidth=2, color='teal', label='stETH yield')
await plot_utils.async_xtick_block_dates()
toolplot.add_tick_grid()
plt.legend()
toolplot.format_yticks(toolstr_kwargs={'percentage': True})
plt.title('Historical Yield For Lido Staking')
plt.show()

## Capital Allocation Model

####  \<Yield and risk parameters>

## FEI Savings Deposit

#### Historical benchmark / basket for FEI Savings Rate yield

In [ ]:
# cFEI
cfei = '0x7713dd9ca933848f6819f38b8352d9a15ea73f67'
cfei_creation_block = await evm.async_get_contract_creation_block(cfei)
cfei_blocks = np.linspace(cfei_creation_block, latest_block_number, n_samples).astype(int)
cfei_yields = await compound_utils.async_get_supply_apy_by_block(ctoken=cfei, blocks=cfei_blocks)

# aFEI
afei_rates = await aave_v2_utils.async_get_interest_rates_by_block(
    token=fei,
    blocks=aave_sample_blocks,
)
afei_yields = adai_rates['supply_apy']

# fFEI (rari fuse)
ffei = '0xd8553552f8868c1ef160eedf031cf0bcf9686945'
ffei_creation_block = await evm.async_get_contract_creation_block(ffei)
ffei_blocks = np.linspace(ffei_creation_block + 450000, latest_block_number, n_samples).astype(int)
coroutines = [
    rari_utils.async_get_supply_apy(ctoken=ffei, block=block)
    for block in ffei_blocks
]
ffei_supply_apy = await asyncio.gather(*coroutines)

In [ ]:
plt.plot(cfei_blocks, cfei_yields, label='cFEI')
plt.plot(aave_sample_blocks, afei_yields, label='aFEI')
plt.plot(ffei_blocks, ffei_supply_apy, label='fFEI')

await plot_utils.async_xtick_block_dates()
toolplot.add_tick_grid()
plt.legend()
toolplot.format_yticks(toolstr_kwargs={'percentage': True})
plt.title('Historical FEI Yield')
plt.show()

## Liquidity Pools

#### Historical FEI Balancer liquidity (Uniswap etc. minor)

In [ ]:
for platform, platform_tvl in fei_analytics_payload['data']['dex_tvls']['metrics'].items():
    plt.plot(fei_analytics_payload['block_numbers'], platform_tvl['values'], label=platform)

await plot_utils.async_xtick_block_dates()
toolplot.add_tick_grid()
plt.legend()
toolplot.format_yticks(toolstr_kwargs={'prefix': '$'})
plt.title('FEI DEX TVL')
plt.show()

## Money Markets

#### Historical Aave, Compound supply / borrow rates and utilization

In [ ]:
# see above